## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-04-11-45-30 +0000,nypost,Ex-husband of NYC teacher in school sex scanda...,https://nypost.com/2025/10/04/us-news/ex-husba...
1,2025-10-04-11-44-34 +0000,nyt,"For the New Archbishop of Canterbury, Nursing ...",https://www.nytimes.com/2025/10/04/world/europ...
2,2025-10-04-11-42-19 +0000,bbc,Starmer tells protesters to 'respect grief of ...,https://www.bbc.com/news/articles/ckgy8kvvkp3o...
3,2025-10-04-11-38-27 +0000,nyt,"Cautious Optimism on Gaza Peace Plan, but No G...",https://www.nytimes.com/2025/10/04/world/middl...
4,2025-10-04-11-33-31 +0000,nyt,Israel and Hamas Say They’ll Work With Trump’s...,https://www.nytimes.com/2025/10/04/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2385/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,53
37,hamas,17
30,gaza,16
316,court,16
177,shutdown,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
84,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...,139
182,2025-10-03-20-03-20 +0000,nypost,Hamas agrees to release all Israeli hostages p...,https://nypost.com/2025/10/03/world-news/hamas...,136
62,2025-10-04-03-10-45 +0000,nypost,"Hamas internal rift over hostages, weapons cou...",https://nypost.com/2025/10/03/world-news/hamas...,126
9,2025-10-04-10-44-52 +0000,wapo,High stakes for Gaza peace talks as Hamas acce...,https://www.washingtonpost.com/world/2025/10/0...,125
124,2025-10-03-22-22-09 +0000,nyt,Trump Calls on Israel to Stop Bombing Gaza Aft...,https://www.nytimes.com/2025/10/03/us/politics...,120


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
84,139,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...
180,62,2025-10-03-20-27-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...
204,60,2025-10-03-18-23-42 +0000,nypost,Terrorist Jihad Al-Shamie was on bail for rape...,https://nypost.com/2025/10/03/world-news/uk-te...
262,58,2025-10-03-14-42-11 +0000,latimes,Supreme Court will decide if gun owners may ca...,https://www.latimes.com/politics/story/2025-10...
24,48,2025-10-04-09-49-00 +0000,wsj,Japan Set for First Female Prime Minister Amid...,https://www.wsj.com/world/asia/japan-prime-min...
143,46,2025-10-03-21-38-40 +0000,nypost,Sean ‘Diddy’ Combs still faces nearly 70 lawsu...,https://nypost.com/2025/10/03/us-news/sean-did...
190,37,2025-10-03-19-37-00 +0000,wsj,Trump ordered another lethal military strike o...,https://www.wsj.com/politics/national-security...
199,34,2025-10-03-18-52-08 +0000,latimes,California gave counties power to inspect ICE ...,https://www.latimes.com/california/story/2025-...
298,32,2025-10-03-12-00-08 +0000,wapo,One victim in Yom Kippur synagogue attack in E...,https://www.washingtonpost.com/world/2025/10/0...
104,32,2025-10-03-23-23-33 +0000,nypost,Andrew Cuomo raises $400K in just 2 days follo...,https://nypost.com/2025/10/03/us-news/andrew-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
